In [30]:
from __future__ import absolute_import, division, print_function

import sys
import os
import warnings

import skimage.io
import skimage.transform
import numpy as np

import pycocotools.coco
import pycocotools.mask

In [31]:
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
new_size = 512  # resize the images to have maximum size 512

image_dir = '/home/ronghang/workspace/DATASETS/coco2014/images/val2014/'
val_file = '/home/ronghang/workspace/DATASETS/coco2014/annotations/instances_val2014.json'
save_image_dir = './coco_val_images/'
save_label_dir = './coco_val_labels_png/'
save_imlist = './data/coco_val.txt'

In [33]:
coco = pycocotools.coco.COCO(val_file)

# Re-map COCO class ids to 1~80 (0 is preserved for background)
coco_cls_ids = coco.getCatIds()
cls_id_map = {coco_cls_ids[n_cls]:n_cls+1 for n_cls in xrange(len(coco_cls_ids))}

loading annotations into memory...
Done (t=10.93s)
creating index...
index created!


In [34]:
# all the image ids
image_ids = coco.getImgIds()
num_im = len(image_ids)

In [35]:
def resize_to_fixed_size(im, new_size):
    # Resize and pad im to input_h x input_w size
    im_h, im_w = im.shape[:2]
    scale = min(new_size * 1.0 / im_h, new_size * 1.0 / im_w)
    resized_h = int(np.round(im_h * scale))
    resized_w = int(np.round(im_w * scale))

    resized_im = skimage.transform.resize(im, [resized_h, resized_w])
    return resized_im

def segToMask(ann, height, width, new_size):
    if type(ann['segmentation']) == list:
        # polygon
        rle = pycocotools.mask.frPyObjects(ann['segmentation'], height, width)
    else:
        # mask
        if type(ann['segmentation']['counts']) == list:
            # uncompressed RLE
            rle = pycocotools.mask.frPyObjects([ann['segmentation']], height, width)
        else:
            # encoded RLE mask
            rle = [ann['segmentation']]

    m = pycocotools.mask.decode(rle)
    m = np.sum(m, axis=2)  # sometimes there are multiple binary map (corresponding to multiple segs)
    m = m.astype(np.bool) # convert to boolean
    
    m = resize_to_fixed_size(m, new_size) > 0
    # compute area
    return m

In [36]:
# Save the image list
with open(save_imlist, 'w') as f:
    for n in xrange(num_im):
        image_id = image_ids[n]
        iminfo = coco.loadImgs(image_id)[0]
        f.write(iminfo['file_name'][:-4]+'\n')

In [37]:
# Save resized images and labels
if not os.path.isdir(save_image_dir): os.mkdir(save_image_dir)
if not os.path.isdir(save_label_dir): os.mkdir(save_label_dir)
warnings.simplefilter("ignore")
for n in xrange(num_im):
    if n % 50 == 0: print('processing %d / %d' % (n, num_im))
    image_id = image_ids[n]
    iminfo = coco.loadImgs(image_id)[0]
    
    # Resize the image to new_size
    im = skimage.io.imread(image_dir+iminfo['file_name'])
    im = resize_to_fixed_size(im, new_size)
    skimage.io.imsave(save_image_dir+iminfo['file_name'], im)
    
    # Load 
    labels = np.zeros((im.shape[0], im.shape[1]), dtype=np.uint8)
    ann_ids = coco.getAnnIds(imgIds=image_id)
    for ann_id in ann_ids:
        ann = coco.loadAnns(ids=ann_id)[0]
        cls_id = cls_id_map[ann['category_id']]
        mask = segToMask(ann, iminfo['height'], iminfo['width'], new_size)
        labels[mask] = cls_id
    
    skimage.io.imsave(save_label_dir+iminfo['file_name'][:-4]+'.png', labels)